In [2]:
import chromedriver_binary
import requests
import time
import pandas as pd


from selenium import webdriver
from bs4 import BeautifulSoup
from tqdm import tqdm

browser = webdriver.Chrome()
browser.get('https://stock-pictures.netlify.app')

### 1. Scrolling images to bottom

In [27]:
for i in tqdm(range(0, 2000000, 1000)):
    browser.execute_script("window.scrollTo(0," + str(i) + ")")
    time.sleep(.1)

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [00:26<00:00,  7.58it/s]


### 2. Scraping Page Source

In [28]:
soup = BeautifulSoup(browser.page_source, 'html.parser')

In [29]:
len(soup.find_all('div', class_ = 'container'))

481

### 3. Scraping images Details and Saving into Data List

In [33]:
data = []

for sp in tqdm(soup.find_all('div', class_ = 'container')):
    img_link = sp.find('img').get('src')
    tags = sp.find('span', class_ = 'tag-color').text[7:].strip()
    likes = int(sp.find('div', class_ = 'likes-comments').find_all('span')[0].text.strip()[:-6])
    comments = int(sp.find('div', class_ = 'likes-comments').find_all('span')[1].text.strip()[:-9])
    
    data.append([img_link, tags, likes, comments])

100%|██████████████████████████████████████████████████████████████████████████████| 481/481 [00:00<00:00, 2367.09it/s]


### 4. Saving images into DataFrame

In [34]:
df = pd.DataFrame(data, columns = ['img_link', 'tags', 'likes', 'comments'])

In [35]:
df

,img_link,tags,likes,comments
0,https://cdn.pixabay.com/photo/2022/03/06/05/30...,"Clouds, Sky, Atmosphere, Blue Sky",196,55
1,https://cdn.pixabay.com/photo/2022/04/07/11/45...,"Bird, Ornithology, Hummingbird",76,20
2,https://cdn.pixabay.com/photo/2022/02/28/15/28...,"Sea, Rainbow, Rainfall, Subtropical",282,106
3,https://cdn.pixabay.com/photo/2022/04/04/02/52...,"Cherry Blossoms, Road, Japan, Sakura",42,11
4,https://cdn.pixabay.com/photo/2022/04/09/18/06...,"Cape Marguerite, Flower, Plant",39,15
...,...,...,...,...
476,https://cdn.pixabay.com/photo/2022/04/07/14/57...,"Horse, Brown Horse, Farm Animal, Equine",4,3
477,https://cdn.pixabay.com/photo/2022/04/07/17/52...,"Flower, Red Flower, Nature, Spring",8,5
478,https://cdn.pixabay.com/photo/2022/04/03/16/47...,"Roses, Bouquet, Flowers, Two-Tone Roses",10,4
479,https://cdn.pixabay.com/photo/2022/04/06/17/15...,"Flowers, Magnolia, Bloom, Blossom",3,0


In [36]:
df = df.drop_duplicates()

In [37]:
df.to_csv('images.csv', index = False)